性別、年齢のみ

In [66]:
import pandas as pd
import numpy as np

# Load the cleaned CSV file (replace 'your_cleaned_file_path.csv' with the actual file path)
file_path_cleaned = r'C:\Users\tora2\vsProject\LifeBehaivor\data\国勢調査2020_日本総人口年性別.csv'
df_cleaned = pd.read_csv(file_path_cleaned, encoding='shift_jis')

# Filter the necessary columns: '年齢', '男女', and 'value'
df_filtered_cleaned = df_cleaned[['年齢', '男女', 'value']].copy()

# Remove any rows that don't have numeric '年齢' values
df_filtered_cleaned = df_filtered_cleaned[df_filtered_cleaned['年齢'] != '総数']

# Convert '年齢' and 'value' columns to appropriate types
df_filtered_cleaned['年齢'] = df_filtered_cleaned['年齢'].str.replace('歳', '').astype(int)
df_filtered_cleaned['value'] = df_filtered_cleaned['value'].astype(int)

# Map the '男女' column to 'MALE' and 'FEMALE'
df_filtered_cleaned['男女'] = df_filtered_cleaned['男女'].map({'男': 'MALE', '女': 'FEMALE'})

# Calculate the total population in the dataset
total_population_cleaned = df_filtered_cleaned['value'].sum()

# Calculate the proportion of each group in the dataset
df_filtered_cleaned['proportion'] = df_filtered_cleaned['value'] / total_population_cleaned

# Generate 100 agents based on these proportions
NUM_AGENTS = 1000
agents_cleaned = []

for _, row in df_filtered_cleaned.iterrows():
    count = int(round(row['proportion'] * NUM_AGENTS))
    agents_cleaned.extend([{'age': row['年齢'], 'gender': row['男女']}] * count)

# If we have slightly fewer or more than 100 agents due to rounding, adjust randomly
while len(agents_cleaned) < NUM_AGENTS:
    agents_cleaned.append(agents_cleaned[np.random.randint(len(agents_cleaned))])
while len(agents_cleaned) > NUM_AGENTS:
    agents_cleaned.pop(np.random.randint(len(agents_cleaned)))

# Convert the list of agents to a DataFrame
agents_cleaned_df = pd.DataFrame(agents_cleaned)

# Output the result to a CSV file
# output_cleaned_path = 'generated_agents_cleaned_final.csv'
# agents_cleaned_df.to_csv(output_cleaned_path, index=False)

# Display the result DataFrame
print(agents_cleaned_df)


     age  gender
0      0    MALE
1      0    MALE
2      0    MALE
3      1    MALE
4      1    MALE
..   ...     ...
995   97  FEMALE
996   91  FEMALE
997   58    MALE
998   72  FEMALE
999   35    MALE

[1000 rows x 2 columns]


初期行動も含める

In [67]:
import pandas as pd
from tqdm import tqdm
import random


DIR_HOURLY = r"C:\Users\tora2\vsProject\LifeBehaivor\data\2020_jikoku_danjonenso.csv"
DIR_GENRE= r"C:\Users\tora2\vsProject\LifeBehaivor\data\nhk_genre.csv"

# データフレームを参照リストに基づいてソートするヘルパー関数
def sort_df_by_list(df, column, reference_list):
    return df.set_index(column).reindex(reference_list).reset_index()


# 独自定義のラベルを適用するためにまず辞書を作る
small = [
    "睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい", 
    "授業・学内の活動", "学校外の学習", "炊事・掃除・洗濯", "買い物", 
    "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際", 
    "スポーツ", "行楽・散策", "趣味・娯楽・教養(インターネット除く)", 
    "趣味・娯楽・教養のインターネット(動画除く)", "インターネット動画", 
    "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本", 
    "音楽", "休息", "その他", "不明"
]
label_master = pd.read_csv(DIR_GENRE)
label_dict = {}
for k,v in zip(label_master["小分類"],label_master["モデル用定義"]):
    label_dict[k]=v

# 行動種別の並び順管理
ACTIVITY_ORDERING = ['睡眠', '食事', '身のまわりの用事', '療養・静養', '仕事', '仕事のつきあい', '授業・学内の活動',
       '学校外の学習', '炊事・掃除・洗濯', '買い物', '子どもの世話', '家庭雑事', '通勤', '通学', '社会参加',
       '会話・交際', 'スポーツ', '行楽・散策', '趣味・娯楽・教養', 'マスメディア接触', '休息']
# label_master["モデル用定義"].unique()

In [68]:
# t_0の行為者割合のtableを作成
# 時間帯ごとの行為者率テーブルの処理
weight_master = {}
for label in tqdm([
        '男１０代', '男２０代', '男３０代', '男４０代', '男５０代', '男６０代', '男７０歳以上',
        '女１０代', '女２０代', '女３０代', '女４０代', '女５０代', '女６０代', '女７０歳以上']):
    # for day in (["平日","土曜日","日曜日"]):
    day = "平日"
    data_hourly = pd.read_csv(DIR_HOURLY)
    data_hourly = data_hourly.set_axis(["Day","Group","Activity","Time","Rate"],axis=1)
    data_hourly = data_hourly.query("Time == '0:00'")
    data_hourly = data_hourly.query("Group == @label & Activity in @small & Day == @day")
    data_hourly["Activity"] = data_hourly["Activity"].map(label_dict) # 独自定義のラベル適用
    data_hourly = data_hourly.groupby(["Time","Day","Group","Activity"],as_index=False).sum().copy()
    weights = []
    for act,r in zip(data_hourly['Activity'],data_hourly["Rate"]):
        weights.append((act,r))
    weight_master[label] = weights


100%|██████████| 14/14 [00:02<00:00,  4.67it/s]


In [69]:
# 各エージェントに行動を割り当てるためのマッピング
# グループと性別、年齢を基にして、適切な行動を割り当てる
def make_label(age:int,gender:str):
    ageStr = ""
    if age >= 10 and age < 20:
        ageStr = "１０代"
    elif age >= 20 and age < 30:
        ageStr = "２０代"
    elif age >= 30 and age < 40:
        ageStr = "３０代"
    elif age >= 40 and age < 50:
        ageStr = "４０代"
    elif age >= 50 and age < 60:
        ageStr = "５０代"
    elif age >= 60 and age < 70:
        ageStr = "６０代"
    else:
        ageStr = "７０歳以上"
    genderJp = ""
    if gender == "MALE":
        genderJp = "男"
    else:
        genderJp = "女"
    
    return genderJp+ageStr

# ルーレット選択を行う関数
def roulette_selection(choices):
    # 数値のリスト（重み）の合計を計算
    total = sum(weight for _, weight in choices)
    
    # ランダムに0からtotalの間の値を選択
    pick = random.uniform(0, total)
    
    # ランダムな選択に対応するアイテムを返す
    current = 0
    for item, weight in choices:
        current += weight
        if current > pick:
            return item


agents_cleaned_df["action"] = agents_cleaned_df.apply(lambda row: roulette_selection(weight_master[make_label(row["age"],row["gender"])]),axis=1)

In [70]:
agents_cleaned_df.to_csv(f"Z:/lab/lifebehavior/agent_{NUM_AGENTS}.csv",index=False)